In [1]:
import numpy as np
import pandas as pd

from sklearn import tree
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/Colab\ Notebooks/stats208

/content/gdrive/MyDrive/Colab Notebooks/stats208


In [ ]:
!ls

boston.csv  ozone.csv


# Load Datasets

In [45]:
ozone = pd.read_csv("ozone.csv").iloc[:, 1:]
boston = pd.read_csv("boston.csv").iloc[:, 1:]

## Ozone

In [50]:
ozone.head()

,Month,DayOfMonth,DayOfYear,MaxDailyOzone,PresHeight,WindSpeed,Humidity,Temp1,InvBaseHeight,PresGrad,InvBaseTemp,Visibility
0,1,3,6,3,5710,4,28,40,2693,-25,47.66,250
1,1,4,7,5,5700,3,37,45,590,-24,55.04,100
2,1,5,1,5,5760,3,51,54,1450,25,57.02,60
3,1,6,2,6,5720,4,69,35,1568,15,53.78,60
4,1,7,3,4,5790,6,19,45,2631,-33,54.14,100


## Boston Housing

In [51]:
boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


# Simulate Datasets

## Fried datasets

In [31]:
def fried1(n, state, p = 10):
  np.random.seed(state)
  X = np.random.uniform(0, 1, size = (n, p))
  eps = np.random.normal(0, 1, n)
  y = 10*np.sin(np.pi*X[:, 0]*X[:, 1]) + 20*(X[:, 2] - 0.5)**2 + 10*X[:, 3] + 5*X[:, 4] + eps

  return X, y

In [34]:
def fried2(n, state, p = 4):
  np.random.seed(state)
  X = np.zeros((n, p))
  X[:, 0] = np.random.uniform(0, 100, n) 
  X[:, 1] = np.random.uniform(40 * np.pi, 560 * np.pi, n)
  X[:, 2] = np.random.uniform(0, 1, n)
  X[:, 3] = np.random.uniform(1, 11, n)

  y = (X[:, 0]**2 + (X[:, 1]*X[:, 2] - (1/(X[:, 1]*X[:, 3])))**2)**0.5
  noise_sig = np.var(y)/3

  y += np.random.normal(0, noise_sig**0.5, n)

  return X, y

In [35]:
def fried3(n, state, p = 4):
  np.random.seed(state)
  X = np.zeros((n, p))
  X[:, 0] = np.random.uniform(0, 100, n) 
  X[:, 1] = np.random.uniform(20, 280, n) * 2 * np.pi
  X[:, 2] = np.random.uniform(0, 1, n)
  X[:, 3] = np.random.uniform(1, 11, n)

  y = np.arctan((X[:, 1]*X[:, 2] - (1/(X[:, 1]*X[:, 3])))/X[:, 0])
  noise_sig = np.var(y)/3

  y += np.random.normal(0, noise_sig**0.5, n)

  return X, y

# Regression Tree

In [12]:
params = {"criterion": ["squared_error", "friedman_mse"], 
          "splitter": ["best", "random"],
          "max_depth": [1, 3, 5, 7, 9, 11, 15]}

In [46]:
def reg_tree(L_x, T_x, L_y, T_y, cv_k = 10):
  #L_x, T_x, L_y, T_y = train_test_split(X, Y, test_size = 0.05)

  dtr = tree.DecisionTreeRegressor()
  opt_mod = RandomizedSearchCV(dtr, param_distributions = params, 
                               scoring = "neg_mean_squared_error", 
                               cv = cv_k, verbose = 0)
  opt_mod.fit(L_x, L_y)

  T_y_pred = opt_mod.predict(T_x)

  mse = mean_squared_error(T_y, T_y_pred)

  return mse

# Bootstrap Regression Tree

In [68]:
def reg_tree_boot(L_x, T_x, L_y, T_y, B, dat_type, cv_k = 10):
    
  #L_x, T_x, L_y, T_y = train_test_split(X, Y, test_size = 0.05, random_state = state)

  n = len(L_x)
  preds = np.zeros((len(T_x), B))


  for i in range(B):
    np.random.seed(i)
    idx = np.random.choice(np.arange(n), n, replace = True)
    if dat_type == "sim":
      Lb_x = L_x[idx, :]
      Lb_y = L_y[idx]
    else: 
      Lb_x = L_x.iloc[idx, :]
      Lb_y = L_y.iloc[idx]

    dtr = tree.DecisionTreeRegressor()
    opt_mod = RandomizedSearchCV(dtr, param_distributions = params, 
                               scoring = "neg_mean_squared_error", 
                               cv = cv_k, verbose = 0)
    opt_mod.fit(Lb_x, Lb_y)
    preds[:, i] = opt_mod.predict(T_x)

  T_y_pred = np.mean(preds, axis = 1)
  mse = mean_squared_error(T_y, T_y_pred)

  return mse

# Experiments

## Boston Housing Results 

Single tree

In [65]:
mse_bos = []

for i in range(100):
  T = boston.sample(n = 25, random_state = i)
  L = boston[~boston.index.isin(T.index)]

  L_x = L.loc[:, L.columns != "medv"]
  L_y = L['medv']

  T_x = T.loc[:, T.columns != "medv"]
  T_y = T["medv"]

  mse_bos.append(reg_tree(L_x, T_x, L_y, T_y))

e_s = np.mean(mse_bos)
print("avg E_S:", e_s)

avg E_S: 25.33985917690343


Bagged Trees

In [69]:
mse_bos_boot = []

for i in range(100):
  T = boston.sample(n = 25, random_state = i)
  L = boston[~boston.index.isin(T.index)]

  L_x = L.loc[:, L.columns != "medv"]
  L_y = L['medv']

  T_x = T.loc[:, T.columns != "medv"]
  T_y = T["medv"]
  mse_bos_boot.append(reg_tree_boot(L_x, T_x, L_y, T_y, 25, "real"))

e_b = np.mean(mse_bos_boot)
print("avg E_B:", e_b)

avg E_B: 11.59995195102736


In [70]:
print("Decrease {:0.0f}%:".format(np.round((e_s - e_b)/e_s * 100)))

Decrease 54%:


## Ozone Results

Single Tree

In [72]:
mse_oz = []

for i in range(100):
  T = ozone.sample(n = 15, random_state = i)
  L = ozone[~ozone.index.isin(T.index)]

  L_x = L.loc[:, L.columns != "MaxDailyOzone"]
  L_y = L["MaxDailyOzone"]

  T_x = T.loc[:, T.columns != "MaxDailyOzone"]
  T_y = T["MaxDailyOzone"]

  mse_oz.append(reg_tree(L_x, T_x, L_y, T_y))

e_s = np.mean(mse_oz)
print("avg E_S:", e_s)

avg E_S: 26.620265539655794


Bagged Trees

In [73]:
mse_oz_boot = []

for i in range(100):
  T = ozone.sample(n = 15, random_state = i)
  L = ozone[~ozone.index.isin(T.index)]

  L_x = L.loc[:, L.columns != "MaxDailyOzone"]
  L_y = L["MaxDailyOzone"]

  T_x = T.loc[:, T.columns != "MaxDailyOzone"]
  T_y = T["MaxDailyOzone"]
  mse_oz_boot.append(reg_tree_boot(L_x, T_x, L_y, T_y, 25, "real"))

e_b = np.mean(mse_oz_boot)
print("avg E_B:", e_b)

avg E_B: 17.51937238600066


In [74]:
print("Decrease {:0.0f}%:".format(np.round((e_s - e_b)/e_s * 100)))

Decrease 34%:


## Friedman 1

Single Tree

In [53]:
mse_f1 = []

for i in range(100):
  L_x1, L_y1 = fried1(200, i)
  T_x1, T_y1 = fried1(1000, i)
  mse_f1.append(reg_tree(L_x1, T_x1, L_y1, T_y1))

e_s = np.mean(mse_f1)
print("avg E_S:", e_s)

avg E_S: 11.36563792759656


Bagged Trees

In [75]:
mse_f1_boot = []

for i in range(100):
  L_x1, L_y1 = fried1(200, i)
  T_x1, T_y1 = fried1(1000, i)
  mse_f1_boot.append(reg_tree_boot(L_x1, T_x1, L_y1, T_y1, 25, "sim"))

e_b = np.mean(mse_f1_boot)
print("avg E_B:", e_b)

avg E_B: 5.661551522700161


In [76]:
print("Decrease {:0.0f}%:".format(np.round((e_s - e_b)/e_s * 100)))

Decrease 79%:


## Friedman 2

Single Tree

In [79]:
mse_f2 = []

for i in range(100):
  L_x2, L_y2 = fried2(200, i)
  T_x2, T_y2 = fried2(1000, i)
  mse_f2.append(reg_tree(L_x2, T_x2, L_y2, T_y2))

print("avg E_S:", np.mean(mse_f2))

avg E_S: 79924.82706225722


Bagged Trees

In [77]:
mse_f2_boot = []

for i in range(100):
  L_x2, L_y2 = fried2(200, i)
  T_x2, T_y2 = fried2(1000, i)
  mse_f2_boot.append(reg_tree_boot(L_x2, T_x2, L_y2, T_y2, 25, "sim"))

e_b = np.mean(mse_f2_boot)
print("avg E_B:", e_b)

avg E_B: 58746.99928340459


## Friedman3

Single Tree

In [60]:
mse_f3 = []

for i in range(100):
  L_x3, L_y3 = fried3(200, i)
  T_x3, T_y3 = fried3(1000, i)
  mse_f3.append(reg_tree(L_x3, T_x3, L_y3, T_y3))

print("avg E_S:", np.mean(mse_f3))

avg E_S: 0.07837688235268922


In [78]:
mse_f3_boot = []

for i in range(100):
  L_x3, L_y3 = fried3(200, i)
  T_x3, T_y3 = fried3(1000, i)
  mse_f3_boot.append(reg_tree_boot(L_x3, T_x3, L_y3, T_y3, 25, "sim"))


print("avg E_B:", np.mean(mse_f3_boot))

avg E_B: 0.04958941133657554
